In [1]:
import ee
import geopandas as gpd
import numpy as np

In [2]:
ee.Authenticate()

Enter verification code: 4/1AX4XfWg_y8u_1IUU_kor3KarIdBO-lmhlhNHYdUleynU5dFui0NIxbN6sLk

Successfully saved authorization token.


In [3]:
ee.Initialize()

In [4]:
# Creating a mask for the study area
df = gpd.read_file("C:/Users/Rasmu/Documents/Thesis/geometry/geometry.shp")
print(df)
coords = []

from shapely.geometry import Point
for index, row in df.iterrows():
     for pt in list(row['geometry'].exterior.coords): 
        coords.append(pt)

print(coords)

geometry = ee.Geometry.Polygon(coords,
                               proj= 'EPSG:4326',
                               geodesic = True,
                               maxError= 1.,
                               evenOdd = False)

   Id                                           geometry
0   0  POLYGON ((7.43810 60.22733, 7.54371 60.22398, ...
[(7.438101242000073, 60.227330023000036), (7.543713227000069, 60.22397726200006), (7.543713227000069, 60.14015822600004), (7.434748480000053, 60.14183460700008), (7.438101242000073, 60.227330023000036)]


In [152]:
# Selecting data
ffa_db = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD')      
                       .filterDate(ee.Date('2020-02-12'), ee.Date('2020-03-15')) 
                       .filterBounds(geometry)
                       .filter(ee.Filter.eq('resolution_meters',10))
                       .select("HV")
                       .first())
ffa_fl = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT') 
                       .filterBounds(geometry) 
                       .filterDate(ee.Date('2020-08-01'), ee.Date('2020-08-31')) 
                       .first()) 
                       #.clip(geometry))

ffa_db.bandNames().getInfo()
#Selection = ffa_db.select('HV')


['HV']

In [155]:
# Selecting data
ffa_db = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(ee.Date('2020-02-12'), ee.Date('2020-03-15')).filterBounds(geometry).filter(ee.Filter.eq('resolution_meters',10)).select("HV")

In [156]:
myCollection = ffa_db

In [157]:
img = ee.Image(myCollection.median())

In [159]:
img.exportToDrive

AttributeError: 'Image' object has no attribute 'exportToDrive'

In [101]:
hist = ffa_fl.select('HV').reduceRegion(
    ee.Reducer.fixedHistogram(0, 0.5, 500),geometry).get('HV').getInfo()

In [98]:
mean = ffa_db.select('HV').reduceRegion(
    ee.Reducer.mean(), geometry).get('HV').getInfo()

In [99]:
mean

-21.764885706969167

In [110]:
# Selecting data
ffa_db = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(ee.Date('2020-02-12'), ee.Date('2020-03-15')).filterBounds(geometry).filter(ee.Filter.eq('resolution_meters',10)).select("HV")

In [140]:
myCollection = ffa_db

In [141]:
img = ee.Image(myCollection.median())

In [142]:
img = img.addBands(ee.Image.pixelLonLat())

In [143]:
img = ee.Image(myCollection.median())

In [144]:
img = img.addBands(ee.Image.pixelLonLat())

In [145]:
img = img.reduceRegion(reducer=ee.Reducer.toList(),\
                                    geometry=geometry,\
                                    maxPixels=1e13,\
                                    scale=10);

In [149]:
print(img)

ee.Dictionary({
  "functionInvocationValue": {
    "functionName": "Image.reduceRegion",
    "arguments": {
      "geometry": {
        "functionInvocationValue": {
          "functionName": "GeometryConstructors.Polygon",
          "arguments": {
            "coordinates": {
              "constantValue": [
                [
                  7.438101242000073,
                  60.227330023000036
                ],
                [
                  7.543713227000069,
                  60.22397726200006
                ],
                [
                  7.543713227000069,
                  60.14015822600004
                ],
                [
                  7.434748480000053,
                  60.14183460700008
                ],
                [
                  7.438101242000073,
                  60.227330023000036
                ]
              ]
            },
            "crs": {
              "functionInvocationValue": {
                "functionName": "Projection"

In [136]:
img.bandNames().getInfo()

AttributeError: 'Dictionary' object has no attribute 'bandNames'

In [137]:
img = img.get('HV')

In [138]:
img

In [139]:
img = ee.Array(img).getInfo()

EEException: Error in map(ID=null):
Image.select: Pattern 'HV' did not match any bands.

In [126]:
data = np.array((ee.Array(img.get("HV")).getInfo()))

EEException: Array: Parameter 'values' is required.

In [118]:
img = img.reduceRegion(reducer=ee.Reducer.toList(),\
                                    geometry=geometry,\
                                    maxPixels=1e13,\
                                    scale=10);

In [119]:
img.bandNames().getInfo()

AttributeError: 'Dictionary' object has no attribute 'bandNames'

In [94]:
# perform any calculation on the image collection here
def anyFunction(img):
    #ndvi = ee.Image(img.normalizedDifference(['B8', 'B4'])).rename(["ndvi"])
    ndvi = ee.Image(img).rename(['HV'])
    return ndvi
 
# export the latitude, longitude and array
def LatLonImg(img):
    img = img.addBands(ee.Image.pixelLonLat())
 
    img = img.reduceRegion(reducer=ee.Reducer.toList(),\
                                        geometry=geometry,\
                                        maxPixels=1e13,\
                                        scale=10);
 
    data = np.array((ee.Array(img.get("HV")).getInfo()))
    lats = np.array((ee.Array(img.get("latitude")).getInfo()))
    lons = np.array((ee.Array(img.get("longitude")).getInfo()))
    return lats, lons, data
 
# covert the lat, lon and array into an image
def toImage(lats,lons,data):
 
    # get the unique coordinates
    uniqueLats = np.unique(lats)
    uniqueLons = np.unique(lons)
 
    # get number of columns and rows from coordinates
    ncols = len(uniqueLons)
    nrows = len(uniqueLats)
 
    # determine pixelsizes
    ys = uniqueLats[1] - uniqueLats[0]
    xs = uniqueLons[1] - uniqueLons[0]
 
    # create an array with dimensions of image
    arr = np.zeros([nrows, ncols], np.float32) #-9999
 
    # fill the array with values
    counter =0
    for y in range(0,len(arr),1):
        for x in range(0,len(arr[0]),1):
            if lats[counter] == uniqueLats[y] and lons[counter] == uniqueLons[x] and counter < len(lats)-1:
                counter+=1
                arr[len(uniqueLats)-1-y,x] = data[counter] # we start from lower left corner
    return arr
 
# map over the image collection
#myCollection  = ffa_db.map(anyFunction)
myCollection = ffa_db
# get the median
result = ee.Image(myCollection.median())#.rename('HV')
#result = myCollection.rename(['result'])
# get the lon, lat and result as 1d array
lat, lon, data = LatLonImg(result)
 
# 1d to 2d array
image  = toImage(lat,lon,data)
 
# in case you want to plot the image
import matplotlib.pyplot as plt
plt.imshow(image)
plt.show()

EEException: Error in map(ID=null):
Image.select: Pattern 'HV' did not match any bands.

In [16]:
Selection.getInfo()

{'type': 'Image',
 'bands': [{'id': 'HV',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [36146, 30633],
   'crs': 'EPSG:32628',
   'crs_transform': [10, 0, 362998.0509339827, 0, -10, 9199999.148691932]}],
 'id': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDH_20200212T083433_20200212T083502_031217_0396FB_A474',
 'version': 1642601881628693,
 'properties': {'GRD_Post_Processing_start': 1581506338130,
  'sliceNumber': 1,
  'GRD_Post_Processing_facility_name': 'Copernicus S1 Core Ground Segment - DPA',
  'resolution': 'H',
  'SLC_Processing_facility_name': 'Copernicus S1 Core Ground Segment - DPA',
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-22.073165399097363, 81.05640036541716],
    [-22.06822274456525, 81.0557995628234],
    [-21.945683498280957, 81.04954693314895],
    [-18.33832969365224, 80.84331232568057],
    [-9.501027551166006, 80.12620742794365],
    [-9.489362157490167, 80.12941013611022],
    [-9.15583198001944, 80.22534179520623],
 

In [17]:
HV = ee.Image(Selection)

In [18]:
band_arrs = HV.sampleRectangle(region=geometry)

In [19]:
band_arr_b4 = band_arrs.get('HV')

In [20]:
np_arr_b4 = np.array(band_arr_b4.getInfo())

EEException: Image.sampleRectangle: Too many pixels in sample; must be <= 262144. Got 974142.

In [ ]:
band_arr_b4

In [ ]:
# get the lat lon and add the ndvi
latlon_list = ee.Image.pixelLonLat().addBands(HV)

In [ ]:
# apply reducer to list
latlon = latlon_list.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=geometry,
  maxPixels=1e8,
  scale=20);

In [ ]:
#print(latlon)

In [ ]:
latlon

In [ ]:
data = np.array((ee.Array(latlon.get('HV')).getInfo()))

In [ ]:
# get the lat lon and add the ndvi
latlon_list = ee.Image.pixelLonLat().addBands(VH)
 
# apply reducer to list
latlon = latlon_list.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=geometry,
  maxPixels=1e8,
  scale=20);
 im1 = ee.Image(im_list.get(0)).select('VV', 'VH').clip(aoi)
 
# get data into three different arrays
data = np.array((ee.Array(latlon.get(VH)).getInfo()))
lats = np.array((ee.Array(latlon.get(latitude)).getInfo()))
lons = np.array((ee.Array(latlon.get(longitude)).getInfo()))
 
# get the unique coordinates
uniqueLats = np.unique(lats)
uniqueLons = np.unique(lons)
 
# get number of columns and rows from coordinates
ncols = len(uniqueLons)    
nrows = len(uniqueLats)

In [ ]:
print(latlon_list)

In [ ]:
array = Selection.toArray(0)

In [ ]:
array